<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Fall 2024<br>
            Dr. Eric Van Dusen <br>
        Shashank Dalmia, Ergun Acikoz</p></td></tr>
</table>

In [ ]:
try: 
    from csaps import csaps
except:
    !pip install csaps
    from csaps import csaps

In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets
from datascience import *
%matplotlib inline
from csaps import csaps

# Cubic Cost Functions

The cubic cost function is one that is commonly used to model a cost function within a firm.  

This notebook uses parameters from a recent article  with some parameters for the cubic cost function, in a journal on Teaching Applied Economics

>"Tractable Cubic Cost Functions for Teaching Microeconomics"
>June 21,2023; Applied Economics Teaching Resources (AETR);
>Scott M. Swinton and Hanzhe Zhang

https://www.aaea.org/UserFiles/file/AETR_2021_003RProofFinal.pdf

- d=3600
- c=177
- b=-15
- a=0.5

Let's use these as starting values but then use widgets to allow us to play with the parameters!

In [ ]:
# Define the variables and parameters
q = sp.symbols('q')  # Quantity of goods produced

# Define the cubic cost function
a = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='a')
b = widgets.FloatSlider(value=-15, min=-30, max=30, step=1, description='b')
c = widgets.FloatSlider(value=177.0, min=0, max=250, step=5, description='c')
d = widgets.FloatSlider(value=3600, min=0, max=10000, step=100, description='d')

def plot_cubic_cost_function(a, b, c, d):
    cost_function = a * q**3 + b * q**2 + c * q + d

    cost_function_np = sp.lambdify(q, cost_function, 'numpy')

    quantity_values = np.linspace(0, 25, 100)
    cost_values = cost_function_np(quantity_values)

    plt.figure(figsize=(8, 6))
    plt.plot(quantity_values, cost_values, label='Cost Function', color='blue')
    plt.xlabel('Quantity Produced')
    plt.ylabel('Cost')
    plt.title('Cubic Cost Function')
    plt.legend()
    plt.grid(True)
    plt.show()

interact(plot_cubic_cost_function, a=a, b=b, c=c, d=d)

##  Great - we can make a cost function that displays the model characteristics that we want 
**Now let's go ahead and make an table from the Function**

We can use Sympy and get a Numpy version of the equation as well 

In [ ]:
q = sp.symbols('q') 
cost_function = a * q**3 + b * q**2 + c * q + d


# You dont know how to do this but we can also convert the SymPy expression to a numpy function
cost_function_np = sp.lambdify(q, cost_function, 'numpy')


Le'ts make the model  by specifying those cubic parameters that we did in the last section 

In [ ]:
d=2000
c=177
b=-15
a=0.5

In [ ]:
quantity_values = np.linspace(0, 30, 16)  # Levels of Q from 0 to 20
quantity_values

In [ ]:
total_costs = cost_function_np(quantity_values)

In [ ]:

cost_table = Table().with_columns(
    'Quantity-Q', quantity_values,
    'Fixed Costs-FC', d,
    'Total Costs-TC', total_costs
)
cost_table

In [ ]:
total_variable_cost = cost_table.column("Total Costs-TC") - cost_table.column('Fixed Costs-FC')
total_variable_cost

In [ ]:
cost_table = cost_table.with_column("Total Variable Cost-TVC", total_variable_cost)
cost_table

In [ ]:
average_total_cost = cost_table.column("Total Costs-TC") / cost_table.column("Quantity-Q")
average_total_cost[0] = 0
average_total_cost


In [ ]:
cost_table = cost_table.with_column("Average Total Cost-ATC", average_total_cost)
cost_table

In [ ]:
average_variable_cost = cost_table.column("Total Variable Cost-TVC") / cost_table.column("Quantity-Q")
average_variable_cost[0] = 0

cost_table = cost_table.with_column("Average Variable Cost-AVC", average_variable_cost)
cost_table

In [ ]:
average_fixed_cost = cost_table.column("Fixed Costs-FC") / cost_table.column("Quantity-Q")
average_fixed_cost[0] = 0

cost_table = cost_table.with_column("Average Fixed Cost-AFC", average_fixed_cost)
cost_table

## And Finally Marginal Costs

In [ ]:
marginal_cost = np.diff(total_costs)
marginal_cost = np.append(make_array(0), marginal_cost)
marginal_cost

In [ ]:
cost_table = cost_table.with_column("Marginal Cost", marginal_cost)
cost_table

In [ ]:
plt.plot(cost_table.column("Quantity-Q"), cost_table.column("Fixed Costs-FC"), marker='o')
plt.plot(cost_table.column("Quantity-Q"), cost_table.column("Total Variable Cost-TVC"), marker='o')
plt.plot(cost_table.column("Quantity-Q"), cost_table.column("Total Costs-TC"), marker='o')
plt.xlabel('Quantity')
plt.ylabel('Cost')
plt.title('TFC, TVC and TC')
plt.legend(make_array("Total Fixed Cost","Total Variable Cost","Total Cost"))

plt.show()

In [ ]:
plt.plot(cost_table.column("Quantity-Q")[1:], cost_table.column("Average Fixed Cost-AFC")[1:], marker='o')
plt.plot(cost_table.column("Quantity-Q")[1:], cost_table.column("Average Variable Cost-AVC")[1:], marker='o')
plt.plot(cost_table.column("Quantity-Q")[1:], cost_table.column("Average Total Cost-ATC")[1:], marker='o')
plt.xlabel('Quantity')
plt.ylabel('Cost')
plt.title('AFC, AVC and ATC')
plt.legend(make_array("Average Fixed Cost","Average Variable Cost","Average Total Cost"))

plt.show()

In [ ]:
# You do not need to understand what the code below is doing. 
output = cost_table.column("Quantity-Q")[1:]
mc = cost_table.column("Marginal Cost")[1:]
avc = cost_table.column("Average Variable Cost-AVC")[1:]
atc = cost_table.column("Average Total Cost-ATC")[1:]

sp_mc = csaps(output, mc, smooth=0.85)
sp_avc = csaps(output, avc, smooth=0.85)
sp_atc = csaps(output, atc, smooth=0.85)

output_s = np.linspace(output.min(), output.max(), 150)
mc_s = sp_mc(output_s)
avc_s = sp_avc(output_s)
atc_s = sp_atc(output_s)

plt.plot(output, mc, marker = 'o', color = 'tab:blue')
plt.plot(output_s, mc_s, alpha=0.7, lw = 2, label='_nolegend_', color = 'tab:blue')
plt.plot(output, avc, marker = 'o', color = 'tab:green')
plt.plot(output_s, avc_s, alpha=0.7, lw = 2, label='_nolegend_', color = 'tab:green')
plt.plot(output, atc, marker = 'o', color = 'tab:orange')
plt.plot(output_s, atc_s, alpha=0.7, lw = 2, label='_nolegend_', color = 'tab:orange')
plt.hlines(y=min(avc), xmin = 11.5, xmax = 13, lw=3, color='r', zorder = 10)
plt.hlines(y=min(atc), xmin = 16, xmax = 18, lw=3, color='r', zorder = 10)
plt.xlabel('Quantity')
plt.ylabel('Cost')
plt.title('MC, AVC and ATC')
plt.legend(make_array("Marginal Cost","Average Variable Cost","Average Total Cost"))

plt.show()